In [64]:
# !pip install torchsummary

In [65]:
"""
Target : Build deep model with many non linearities(ReLU layers) for base model structure
Result : 104,400 params, test: 99.23%, train 99.35% at 14th
Analysis:  very slow training 
"""

'\nTarget : base model structure\nResult : test: 99.02%, train 98.8\nAnalysis: delayed training, very huge parameter\n'

In [66]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from model import Model_1

In [68]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Model_1().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
            Conv2d-3           [-1, 32, 24, 24]           4,608
              ReLU-4           [-1, 32, 24, 24]               0
            Conv2d-5           [-1, 64, 22, 22]          18,432
              ReLU-6           [-1, 64, 22, 22]               0
         MaxPool2d-7           [-1, 64, 11, 11]               0
            Conv2d-8           [-1, 16, 11, 11]           9,216
              ReLU-9           [-1, 16, 11, 11]               0
           Conv2d-10           [-1, 32, 11, 11]           4,608
             ReLU-11           [-1, 32, 11, 11]               0
           Conv2d-12             [-1, 64, 9, 9]          18,432
           Conv2d-13             [-1, 16, 9, 9]           9,216
             ReLU-14             [-1, 1

/var/folders/m2/x6hw0bg17hd2xt6rzl4jd04h0000gn/T/ipykernel_50973/542158296.py:58: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)


In [69]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 7, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [70]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct=0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.float().to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        pred = output.argmax(dim=1)
        correct+=pred.eq(target).sum().item()
    acc = 100*correct/len(train_loader.dataset)
    print(f'Train Accuracy: {acc:.2f}%')
        
def test(model, device, test_loader,epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for _,(data, target) in enumerate(test_loader):
            data, target = data.float().to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [72]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
for epoch in range(1, 15):
    print(f'Epoch: {epoch}')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader,epoch)


Epoch: 1


  0%|          | 0/938 [00:00<?, ?it/s]/var/folders/m2/x6hw0bg17hd2xt6rzl4jd04h0000gn/T/ipykernel_50973/542158296.py:58: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)
loss=2.302572727203369 batch_id=937: 100%|██████████| 938/938 [01:09<00:00, 13.45it/s] 


Train Accuracy: 11.68%
Test set: Average loss: 2.3026, Accuracy: 1311/10000 (13.11%)
Epoch: 2


loss=2.302574634552002 batch_id=937: 100%|██████████| 938/938 [01:07<00:00, 13.80it/s] 


Train Accuracy: 14.56%
Test set: Average loss: 2.3026, Accuracy: 1498/10000 (14.98%)
Epoch: 3


...

Train Accuracy: 99.08%
Test set: Average loss: 0.0818, Accuracy: 9772/10000 (97.72%)
Epoch: 13


loss=0.03821629658341408 batch_id=937: 100%|██████████| 938/938 [01:02<00:00, 15.09it/s]   


Train Accuracy: 99.14%
Test set: Average loss: 0.0357, Accuracy: 9894/10000 (98.94%)
Epoch: 14


loss=0.07509544491767883 batch_id=937: 100%|██████████| 938/938 [01:00<00:00, 15.60it/s]   


Train Accuracy: 99.28%
Test set: Average loss: 0.0294, Accuracy: 9907/10000 (99.07%)
